## 1. Benchmark Dataset: 
타이타닉 호에서 탑승했던 사람들의 정보를 바탕으로 생존 여부를 예측
하는 문제입니다. 과제에서 제시한 머신러닝 모델들을 이용하여 각 모델의 성능을 평가해야
합니다. 데이터는 다음 주소에서 얻을 수 있습니다: https://www.kaggle.com/c/titanic
이번 과제에서는 모델 학습 및 테스트 모두 train.csv파일을 이용합니다.
과제를 수행하실 때 다음의 feature들은 모델 학습에 이용하지 않도록 주의해주세요.
제외할 feature: PassengerId, Name, Ticket, Cabin

## 2. Preprocessing
    1) train data에 결측치(NULL)가 있는 feature들이 있습니다. 이 값들을 어떻게 처리할 것인지 아이디어를 제시하고 실제로 구현하세요. 
    2) One-Hot encoding을 수행할 필요가 있는 feature들이 무엇이며 그 이유는 무엇인지 서술하세요. 또한 실제로 어떻게 구현했는지 보고서에 나타내세요.
    3) train.csv의 sample을 7대 3으로 학습 데이터와 테스트 데이터로 사용하세요.
    4) 그 외 진행한 전처리 과정이 있다면 서술하세요

https://dlearner.tistory.com/22

#### 가장 먼저, 2-3) train.csv 파일 안의 891개 sample 중에서 89 * 3 = 267개를 떼어내어 tran.csv를 만들었음

In [309]:
import pandas as pd
import numpy as np
import random as rnd

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

In [291]:
# 모델
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import BaggingClassifier

In [586]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv('test.csv')
combine=[train_df, test_df]

#### 1 제외 할 feature: PassengerId, Name, Ticket, Cabin

In [587]:
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
combine=[train_df, test_df]

In [588]:
print(train_df.columns.values)

['Survived' 'Pclass' 'Sex' 'Age' 'SibSp' 'Parch' 'Fare' 'Embarked']


#### 2-1) train data에 결측치(NULL)가 있는 feature들이 있습니다. 이 값들을 어떻게 처리할 것인지 아이디어를 제시하고 실제로 구현하세요.
<br>  -> 'Age' column에선 다수의, 'Embarked' column에선 극소수의 샘플에 대해 NULL값이 있음. 
<br>  a) 'Embarked'는 (눈으로 세어보니 2개) 워낙 갯수도 적고, 생존 여부에 영향을 주는 attribute이면서도 평균이나 중간값 등으로 처리하기 애매하므로 그냥 해당 sample은 drop. 
<br>  b) 'Age'에 column에서 나오는 NULL 갯수는 무시할 수 없을 만큼 많은데, 그렇다고 싹다 평균이나 중간값으로 대체하는 것은 실제 데이터(기록되진 않았지만 그 사람의 실제 나이)와는 괴리가 너무 커질 것으로 예상. 따라서 NULL값은 최빈값처럼 취급하고, 'Parch가 0인데 Pclass가 1이면 나이를 중장년층으로 예상' 같은 규칙을 추가해 처리하고자 함


#### a)부터 해결 후 b)의 경우는 2-4)에서 다시 처리 예정

In [589]:
#train_df = train_df.dropna(subset=['Embarked'])
#test_df = test_df.dropna(subset=['Embarked'])
combine=[train_df, test_df]

#### 2-2) One-Hot encoding을 수행할 필요가 있는 feature들이 무엇이며 그 이유는 무엇인지 서술하세요. 또한 실제로 어떻게 구현했는지 보고서에 나타내세요.
-> scikit-learn을 사용할 때 문자열로 나타나는 feature들은 Label이든 One-Hot이든 숫자로 encoding해 줘야 함. 'Sex', 'Embarked' 을 대상으로 One-Hot encoding이 필요해 보여 이를 수행

In [590]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
OHE = OneHotEncoder(sparse=False)
LE = LabelEncoder()

In [591]:
print(combine[0].head(65))
print(combine[0])

    Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0          0       3    male  22.0      1      0   7.2500        S
1          1       1  female  38.0      1      0  71.2833        C
2          1       3  female  26.0      0      0   7.9250        S
3          1       1  female  35.0      1      0  53.1000        S
4          0       3    male  35.0      0      0   8.0500        S
..       ...     ...     ...   ...    ...    ...      ...      ...
60         0       3    male  22.0      0      0   7.2292        C
61         1       1  female  38.0      0      0  80.0000      NaN
62         0       1    male  45.0      1      0  83.4750        S
63         0       3    male   4.0      3      2  27.9000        S
64         0       1    male   NaN      0      0  27.7208        C

[65 rows x 8 columns]
     Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0           0       3    male  22.0      1      0   7.2500        S
1           1       1  female  38.0  

In [592]:
print(train_df['Sex'].head(64) )

0       male
1     female
2     female
3     female
4       male
       ...  
59      male
60      male
61    female
62      male
63      male
Name: Sex, Length: 64, dtype: object


In [585]:
for dataset in combine :
    dataset['Sex'] = dataset['Sex'].map({'female':1, 'male':0}).astype(int)

for dataset in combine :
    dataset['Embarked'] = dataset['Embarked'].map({'S':1, 'Q':2, 'C': 3}).astype(int)


NameError: name 'NA' is not defined

https://ordo.tistory.com/51
    https://wikidocs.net/29530
    https://workingwithpython.com/howtohandlemissingvaluewithpython/

In [567]:
OHE.fit(train_df[['Sex']])
print(train_df[['Sex']].head(64) )

    Sex
0     0
1     1
2     1
3     1
4     0
..  ...
59    0
60    0
62    0
63    0
64    0

[64 rows x 1 columns]


In [568]:
#train_df 재정비

OHE.fit(train_df[['Sex']])
train_tmpSex = pd.DataFrame( OHE.transform(train_df[['Sex']]), columns=['Sex_M', 'Sex_F'])
OHE.fit(train_df[['Embarked']])
train_tmpEmbarked = pd.DataFrame( OHE.transform(train_df[['Embarked']]), columns=['Embarked_S', 'Embarked_Q', 'Embarked_C'])



train_tmp = pd.concat([train_tmpSex, train_tmpEmbarked], axis=1)
train_tmp = train_tmp.dropna(subset=['Embarked_S'])

#train_df = pd.concat([train_df, train_tmp], axis=1)

train_df = pd.concat([train_df, train_tmpSex, train_tmpEmbarked], axis=1)

#print(train_tmp.head(64))
print(train_df.head(64))

#train_df = pd.concat([train_df, train_tmpSex, train_tmpEmbarked], axis=1)

#train_df = train_df.drop(['Sex', 'Embarked'], axis=1)

#test_df  재정비

OHE.fit(test_df[['Sex']])
test_tmpSex = pd.DataFrame( OHE.transform(test_df[['Sex']]), columns=['Sex_M', 'Sex_F'])
OHE.fit(test_df[['Embarked']])
test_tmpEmbarked = pd.DataFrame( OHE.transform(test_df[['Embarked']]), columns=['Embarked_S', 'Embarked_Q', 'Embarked_C'])


#test_df = pd.concat([test_df, test_tmpSex, test_tmpEmbarked], axis=1)
#test_df = test_df.drop(['Sex', 'Embarked'], axis=1)
test_tmpEmbarked = test_tmpEmbarked.dropna(subset=['Embarked_S'])


combine=[train_df, test_df]

    Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  Sex_M  \
0        0.0     3.0  0.0  22.0    1.0    0.0   7.2500       1.0    1.0   
1        1.0     1.0  1.0  38.0    1.0    0.0  71.2833       3.0    0.0   
2        1.0     3.0  1.0  26.0    0.0    0.0   7.9250       1.0    0.0   
3        1.0     1.0  1.0  35.0    1.0    0.0  53.1000       1.0    0.0   
4        0.0     3.0  0.0  35.0    0.0    0.0   8.0500       1.0    1.0   
..       ...     ...  ...   ...    ...    ...      ...       ...    ...   
59       0.0     3.0  0.0  11.0    5.0    2.0  46.9000       1.0    1.0   
60       0.0     3.0  0.0  22.0    0.0    0.0   7.2292       3.0    1.0   
61       NaN     NaN  NaN   NaN    NaN    NaN      NaN       NaN    1.0   
62       0.0     1.0  0.0  45.0    1.0    0.0  83.4750       1.0    1.0   
63       0.0     3.0  0.0   4.0    3.0    2.0  27.9000       1.0    1.0   

    Sex_F  Embarked_S  Embarked_Q  Embarked_C  
0     0.0         1.0         0.0         0.0  
1  

In [503]:
print(train_df.head(64))
print(train_tmpSex.head(64))
print(train_tmpEmbarked.head(64))

    Survived  Pclass   Age  SibSp  Parch     Fare
0          0       3  22.0      1      0   7.2500
1          1       1  38.0      1      0  71.2833
2          1       3  26.0      0      0   7.9250
3          1       1  35.0      1      0  53.1000
4          0       3  35.0      0      0   8.0500
..       ...     ...   ...    ...    ...      ...
59         0       3  11.0      5      2  46.9000
60         0       3  22.0      0      0   7.2292
62         0       1  45.0      1      0  83.4750
63         0       3   4.0      3      2  27.9000
64         0       1   NaN      0      0  27.7208

[64 rows x 6 columns]
    Sex_M  Sex_F
0     1.0    0.0
1     0.0    1.0
2     0.0    1.0
3     0.0    1.0
4     1.0    0.0
..    ...    ...
59    1.0    0.0
60    1.0    0.0
61    1.0    0.0
62    1.0    0.0
63    1.0    0.0

[64 rows x 2 columns]
    Embarked_S  Embarked_Q  Embarked_C
0          1.0         0.0         0.0
1          0.0         0.0         1.0
2          1.0         0.0       

In [350]:
test_df = pd.concat([test_df, test_tmpSex, test_tmpEmbarked])


In [258]:

train_tmpSex = LE.fit_transform(train_df['Sex'])
test_tmpSex = LE.fit_transform(test_df['Sex'])

train_tmpEmbarked = LE.fit_transform(train_df[['Embarked']])
test_tmpEmbarked = LE.fit_transform(test_df[['Embarked']])

print(train_tmpEmbarked)
print(train_tmpEmbarked.categories)



[0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 1 0 1 1 1 0 1 0 1
 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 1 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0
 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0
 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1
 0 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 1 1 1
 0 0 1 0 1 0 1 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 0 0 0 1 

AttributeError: 'numpy.ndarray' object has no attribute 'categories'

In [249]:
train_df.head(30)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1.0,22.0,1,0,7.2500,1.0
1,1,1,0.0,38.0,1,0,71.2833,0.0
2,1,3,0.0,26.0,0,0,7.9250,1.0
3,1,1,0.0,35.0,1,0,53.1000,1.0
4,0,3,1.0,35.0,0,0,8.0500,1.0
5,0,3,1.0,NaN,0,0,8.4583,0.0
6,0,1,1.0,54.0,0,0,51.8625,1.0
7,0,3,1.0,2.0,3,1,21.0750,1.0
8,1,3,0.0,27.0,0,2,11.1333,1.0
9,1,2,0.0,14.0,1,0,30.0708,0.0


In [253]:
train_tmpSex = OHE.fit_transform(train_df[['Sex']]).toarray()
test_tmpSex = OHE.fit_transform(test_df[['Sex']]).toarray()

train_tmpEmbarked = OHE.fit_transform(train_df[['Embarked']]).toarray()
test_tmpEmbarked = OHE.fit_transform(test_df[['Embarked']]).toarray()

print(train_tmpEmbarked)
print(train_tmpEmbarked)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]
[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [255]:
train_df['Sex'] = train_tmpSex
test_df['Sex'] = test_tmpSex

train_df['Embarked'] = train_tmpEmbarked
test_df['Embarked'] = test_tmpEmbarked

print(train_tmpEmbarked)
print(train_df['Embarked'])

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]
0      0.0
1      1.0
2      0.0
3      0.0
4      0.0
      ... 
619    0.0
620    1.0
621    0.0
622    1.0
623    0.0
Name: Embarked, Length: 623, dtype: float64


In [ ]:
train_df.head(64)

  #### 2-4) 그 외 진행한 전처리 과정이 있다면 서술하세요
    -> 범주에 따른 분류가 가능해야 하므로, 원래는 연속적 성질을 가진 feature인 'Age'와 'Fare'를 일정 구간 별로 나누어 범주화시켰음, 2-1-b)를 해결하고 OneHotEncoding까지

## 3. Machine Learning Models: scikit-learn을 이용해 후술할 세 가지 machine learning model를 구현하고 성능을 평가하세요.

3-1 K-Nearest Neighbors(KNN): K(이웃)의 개수를 [1~5]까지 변화시키면서 test data에서 결과가
어떻게 변하는지 분석하세요

4. Evaluation Methods: 각각 모델에 따른 성능을 Accuracy, F1-Score을 통해 나타내세요.


3-2 Logistic Regression: Iteration 횟수를 [0~100] 범위에서 20씩 변화시키면서 test data에서 결
과가 어떻게 변하는지 분석하세요. Iteration 횟수를 100으로 고정한 후 regularization
term(scikit-learn에서는 C)를 [0~5]의 범위에서 1씩 변화시키면서 test data에서 결과가 어떻게
변하는지 분석하세요.

4. Evaluation Methods: 각각 모델에 따른 성능을 Accuracy, F1-Score을 통해 나타내세요.

3-3 Decision Tree: Information Gain을 통해 test data에서 결과가 어떻게 나오는지 분석하세요.
또한 적절한 tool을 이용하여 각 depth에서의 조건과 gain값을 알 수 있도록 tree를 시각화
하세요.

4. Evaluation Methods: 각각 모델에 따른 성능을 Accuracy, F1-Score을 통해 나타내세요.

3-3-1 Bagging with Decision Tree: Decision Tree 기반으로 Bagging 기법을 이용하여 bag의
수에 따라 test data에서 결과가 어떻게 변하는지 decision tree와 비교하여 분석하세요.
Bag의 수는 [1~5]에서 1씩 변화시키세요

4. Evaluation Methods: 각각 모델에 따른 성능을 Accuracy, F1-Score을 통해 나타내세요.